In [1]:
import pandas as pd, numpy as np
import itertools, requests, json

In [2]:
dm=pd.read_excel('data/szekelyhon_medve.xlsx')
dc=pd.read_excel('data/szekelyhon_medve_curated.xlsx')
lm=pd.read_excel('data/szekelyhon_locs_curated.xlsx')

In [3]:
dc=dc[(dc['duplicate']!=1)&(dc['relevant']==1)]

In [4]:
dc["uid"] = dc["title"].map(str) + dc["content"]
dm["uid"] = dm["title"].map(str) + dm["content"]
lm["uid"] = lm["title"].map(str) + lm["content"]

In [5]:
dc=dc.set_index('uid')[['relevant','severity','deaths','duplicate']]
dm=dm.set_index('uid')
lm=lm.set_index('uid')[['predicted_locs','actual_locs']]

In [6]:
dmi=dm.loc[dc.index.dropna()]

In [7]:
df=dmi.join(dc).drop_duplicates()

In [8]:
dl=df.join(lm)

In [9]:
len(dl)

10

Working df ready. Now infer location.

In [10]:
to_geo=[i.split(',') for i in dl['actual_locs'].unique() if str(i)!='nan']

In [11]:
to_geo=[i.strip() for i in list(itertools.chain.from_iterable(to_geo))]

Load HU names

In [12]:
coords=json.loads(open('geo/huncoords.json','r').read())
megyek=json.loads(open('geo/megyek.json','r').read())

In [13]:
data=[]
header=['index']
for i in dl.iteritems():
    header.append(i[0])
header.append('geo_loc')
header.append('latitude')
header.append('longitude')
header.append('id')
for i in dl.T.iteritems():
    dummy=[i[0]]
    for j in range(len(i[1])):
        if j==(len(i[1])-1):
            dummy.append(i[1][j])
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dummy.append('')
            dtime=str(dummy[1])[:10]
            if dtime=='nan': dtime=u'N/A' 
            if str(i[1][j])!='nan':    
                for k in i[1][j].split(','):
                    tk=k.strip().replace('[','').replace(']','').replace("'",'')
                    dummy[-4]=tk
                    if tk in coords:
                        dummy[-3]=coords[tk][0]
                        dummy[-2]=coords[tk][1]
                    else:
                        dummy[-3]=''
                        dummy[-2]=''
                    #dummy[-1]=str(dummy[-3])+' ('+str(dummy[0]+1)+')\n'+dtime
                    dummy[-1]=str(dummy[-4])+', '+dtime
                    data.append(list(dummy))                    
            else:
                dummy[-1]=u'N/A' +', '+dtime
                data.append(list(dummy))
        else: 
            dummy.append(i[1][j])

In [14]:
df=pd.DataFrame(data)
df.columns=header
df

,index,Unnamed: 0,date,hudate,link,image,category,icategory,tags,title,...,relevant,severity,deaths,duplicate,predicted_locs,actual_locs,geo_loc,latitude,longitude,id
0,Feldúlva találták a sírhelyeketFeldúlva talált...,0,2019-12-25,2019. december 25.,https://szekelyhon.ro/aktualis/feldulva-talalt...,https://media.szekelyhon.ro/pictures/agerpres/...,aktualis,agerpres,"['Gyergyószék', 'medve']",Feldúlva találták a sírhelyeket,...,1,1,0,0,"['Vasláb', 'Heveder']",Vasláb,Vasláb,46.646075,25.623711,"Vasláb, 0"
1,Elutasította medveügyben a Zetelaki Területtul...,1,2019-12-26,2019. december 26.,https://szekelyhon.ro/aktualis/elutasitotta-me...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Hargita megye', 'medve']",Elutasította medveügyben a Zetelaki Területtul...,...,1,0,0,0,"['Marosvásárhely', 'Zetelaka']",Zetelaka,Zetelaka,46.391031,25.367929,"Zetelaka, 1"
2,"Ki akadályozta meg, hogy a barnamedve lekerülj...",3,2020-01-04,2020. január 04.,https://szekelyhon.ro/aktualis/ki-akadalyozta-...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Belföld', 'medve', 'RMDSZ']","Ki akadályozta meg, hogy a barnamedve lekerülj...",...,1,0,0,0,NaN,Sepsiszentgyörgy,Sepsiszentgyörgy,45.860937,25.788580,"Sepsiszentgyörgy, 3"
3,Pénzt ígér a miniszter a medvék által veszélye...,4,2020-01-06,2020. január 06.,https://szekelyhon.ro/aktualis/penzt-iger-a-mi...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Háromszék', 'medve']",Pénzt ígér a miniszter a medvék által veszélye...,...,1,0,0,0,Zabola,Zabola,Zabola,45.887703,26.189789,"Zabola, 4"
4,Hargita megye: megvan az év első 112-s medveés...,5,2020-01-11,2020. január 11.,https://szekelyhon.ro/aktualis/hargita-megye-m...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Csendőrség', 'medve', 'Parajd']",Hargita megye: megvan az év első 112-s medveés...,...,1,2,0,0,Parajd,Parajd,Parajd,46.555343,25.127272,"Parajd, 5"
5,Aktívak a medvék Székelyudvarhely környékénNoh...,6,2020-01-13,2020. január 13.,https://szekelyhon.ro/aktualis/aktivak-a-medve...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Udvarhelyszék']",Aktívak a medvék Székelyudvarhely környékén,...,1,0,0,0,Székelyudvarhely,Székelyudvarhely,Székelyudvarhely,46.305000,25.292648,"Székelyudvarhely, 6"
6,Lemondott a vadásztársaság az emberre támadó h...,7,2020-01-16,2020. január 16.,https://szekelyhon.ro/aktualis/lemondott-a-vad...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['medve', 'Udvarhelyszék']",Lemondott a vadásztársaság az emberre támadó h...,...,1,0,0,0,Hidegkút,Hidegkút,Hidegkút,46.335918,24.894143,"Hidegkút, 7"
7,"Amíg elérhető közelségben van az ételmaradék, ...",8,2020-01-21,2020. január 21.,https://szekelyhon.ro/aktualis/szagra-jonnek-a...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['medve', 'Székelyudvarhely']","Amíg elérhető közelségben van az ételmaradék, ...",...,1,0,0,0,Székelyudvarhely,Székelyudvarhely,Székelyudvarhely,46.305000,25.292648,"Székelyudvarhely, 8"
8,Medveradar: Zetelakán és Farkaslakán voltak a ...,9,2020-01-26,2020. január 26.,https://szekelyhon.ro/aktualis/medveradar-zete...,https://media.szekelyhon.ro/pictures/udvarhely...,aktualis,udvarhely,"['Hargita megye', 'medve']",Medveradar: Zetelakán és Farkaslakán voltak a ...,...,1,0,0,0,"['Haraly', 'Zetelaka', 'Farkaslaka']",Csíkszereda,Csíkszereda,46.369557,25.795365,"Csíkszereda, 9"
9,Gyergyószentmiklós utcáin kóborolt egy medveEg...,10,2020-01-28,2020. január 28.,https://szekelyhon.ro/aktualis/gyergyoszentmik...,https://media.szekelyhon.ro/pictures/csik/aktu...,aktualis,csik,"['Gyergyószentmiklós', 'medve']",Gyergyószentmiklós utcáin kóborolt egy medve,...,1,2,0,0,Gyergyószentmiklós,Gyergyószentmiklós,Gyergyószentmiklós,46.721211,25.585528,"Gyergyószentmiklós, 10"


Save data for loading in viz app

In [15]:
df['checked']=''
df['exact_loc']=0
df['source']='Székelyhon'

In [16]:
df=df[['checked',
    'date',
    'link',
    'image',
    'source',
    'title',
    'content',
    'relevant',
    'severity',
    'deaths',
    'duplicate',
    'geo_loc',
    'exact_loc',
    'latitude',
    'longitude']].set_index('checked')

In [17]:
#PowerBI exploration data
df.to_excel('data/szekelyhon_data.xlsx')